In [ ]:
# === Embodied Life / HighLife + Diffusive Food — Manual LWSS Injection ===
# Manual mode: inject LWSS (or another pattern) at t=0, centered; no random seeding/spawn.

import numpy as np
import pygame
import sys

# -----------------------------
# 0) Hyperparameters
# -----------------------------
CFG = dict(
    # Cellular rule
    mode="normal",            # "normal" (B3/S23) | "high" (B36/S23)

    # Gate cadence (hard kill if local food < 0.1 for CURRENTLY-ALIVE cells)
    gate_every=10,

    # Manual pattern injection
    manual_mode=False,         # True => no random noise; inject chosen pattern at t=0
    pattern_name="lwss",      # "lwss" | "glider" | "blinker" | "block"
    manual_F0=1.0,            # constant food level in manual mode

    # Grid & timing
    H=400, W=600,
    dt=0.5,
    ticks_per_render=10,
    target_fps=60,

    # Food (kept as in your config; disabled during manual_mode)
    F_min=0.0, F_max=1.0,
    F_init_prob=1,
    food_spawn_prob=7e-5,
    food_spawn_value=1.0,

    # Diffusion (no advection)
    D_dead=0.004, D_alive=0.5,
    use_harmonic_faces=True,
    enforce_cfl_assert=True,

    # Consumption + tiny leak
    mu=0.04,
    lambda_leak=1e-4,

    # Food measurement for gate
    gate_on_local_meanF=True,

    # Central seeding (disabled in manual_mode)
    seed_zone_D=6,
    p_seed_base=2e-2,
    warmup_ticks=10,
    p_seed_warmup=3e-3,

    # Visualization
    cell_px=2,
    color_live=(255, 255, 255),
    show_food_overlay=True,

    # Logging
    log_every_ticks=60,
)

# -----------------------------
# 1) Utilities & kernels
# -----------------------------
rng = np.random.default_rng()

H, W = CFG['H'], CFG['W']
dt = CFG['dt']

def neigh8_sum(C: np.ndarray) -> np.ndarray:
    up    = np.roll(C, -1, axis=0)
    down  = np.roll(C,  1, axis=0)
    left  = np.roll(C,  1, axis=1)
    right = np.roll(C, -1, axis=1)
    up_l    = np.roll(up,    1, axis=1)
    up_r    = np.roll(up,   -1, axis=1)
    down_l  = np.roll(down,  1, axis=1)
    down_r  = np.roll(down, -1, axis=1)
    return (up + down + left + right + up_l + up_r + down_l + down_r).astype(np.int16)

def mean3x3(X: np.ndarray) -> np.ndarray:
    up    = np.roll(X, -1, axis=0)
    down  = np.roll(X,  1, axis=0)
    left  = np.roll(X,  1, axis=1)
    right = np.roll(X, -1, axis=1)
    up_l    = np.roll(up,    1, axis=1)
    up_r    = np.roll(up,   -1, axis=1)
    down_l  = np.roll(down,  1, axis=1)
    down_r  = np.roll(down, -1, axis=1)
    s = (X + up + down + left + right + up_l + up_r + down_l + down_r).astype(np.float32)
    return s * (1.0/9.0)

def hmean(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    return (2.0 * a * b) / (a + b + 1e-12)

def build_rule_LUTs(mode: str):
    mode = (mode or "normal").lower().strip()
    if mode not in ("normal", "high"):
        mode = "normal"
    S = {2, 3}
    B = {3} if mode == "normal" else {3, 6}
    alive_lut = np.zeros(9, dtype=np.uint8)
    dead_lut  = np.zeros(9, dtype=np.uint8)
    for k in range(9):
        alive_lut[k] = 1 if k in S else 0
        dead_lut[k]  = 1 if k in B else 0
    return alive_lut, dead_lut

alive_lut, dead_lut = build_rule_LUTs(CFG["mode"])

def assert_cfl(Dx: np.ndarray, Dy: np.ndarray, dt: float):
    if not CFG['enforce_cfl_assert']:
        return
    D_face_max = float(max(Dx.max(initial=0.0), Dy.max(initial=0.0)))
    if D_face_max <= 0.0:
        return
    limit = 1.0 / (4.0 * D_face_max)
    if dt > limit + 1e-12:
        raise RuntimeError(f"CFL violated: dt={dt:.6f} > 1/(4*Dmax)={limit:.6f} (Dmax_face={D_face_max:.6f})")

def build_frame_surface(C: np.ndarray, F: np.ndarray) -> pygame.Surface:
    G = (np.clip(F, 0.0, 1.0) * 255.0).astype(np.uint8)
    rgb = np.zeros((H, W, 3), dtype=np.uint8)
    if CFG['show_food_overlay']:
        rgb[..., 1] = G
    rgb[C == 1] = CFG['color_live']
    surf = pygame.surfarray.make_surface(np.transpose(rgb, (1, 0, 2)))
    if CFG['cell_px'] != 1:
        surf = pygame.transform.scale(surf, (W*CFG['cell_px'], H*CFG['cell_px']))
    return surf

# -----------------------------
# 2) Patterns (binary arrays)
# -----------------------------
def pattern_lwss() -> np.ndarray:
    """
    Lightweight Spaceship (LWSS), 5x4, orientation that moves right in Life.
    RLE decoded: x=5,y=4:  o2bo$4bo$o3bo$b4o!
    Rows (0=dead,1=live):
      [1,0,0,1,0]
      [0,0,0,0,1]
      [1,0,0,0,1]
      [0,1,1,1,1]
    """
    return np.array([
        [1,0,0,1,0],
        [0,0,0,0,1],
        [1,0,0,0,1],
        [0,1,1,1,1],
    ], dtype=np.uint8)

def pattern_glider() -> np.ndarray:
    return np.array([[0,1,0],
                     [0,0,1],
                     [1,1,1]], dtype=np.uint8)

def pattern_blinker() -> np.ndarray:
    return np.array([[1,1,1]], dtype=np.uint8)

def pattern_block() -> np.ndarray:
    return np.array([[1,1],
                     [1,1]], dtype=np.uint8)

PATTERNS = {
    "lwss":    pattern_lwss,
    "glider":  pattern_glider,
    "blinker": pattern_blinker,
    "block":   pattern_block,
}

def insert_pattern_center(C: np.ndarray, P: np.ndarray) -> None:
    ph, pw = P.shape
    r0 = H//2 - ph//2
    c0 = W//2 - pw//2
    C[r0:r0+ph, c0:c0+pw] |= P

# -----------------------------
# 3) Initialization
# -----------------------------
C = np.zeros((H, W), dtype=np.uint8)

if CFG["manual_mode"]:
    F = np.full((H, W), float(CFG["manual_F0"]), dtype=np.float32)
else:
    F = (rng.random((H, W)) < CFG['F_init_prob']).astype(np.float32) * CFG['food_spawn_value']
    F = np.clip(F, CFG['F_min'], CFG['F_max']).astype(np.float32)

if CFG["manual_mode"]:
    P = PATTERNS.get(CFG["pattern_name"].lower().strip(), pattern_lwss)()
    insert_pattern_center(C, P)

# -----------------------------
# 4) Pygame setup
# -----------------------------
pygame.init()
pygame.display.set_caption("Embodied Life/HighLife + Diffusive Food — Manual LWSS")
screen = pygame.display.set_mode((W*CFG['cell_px'], H*CFG['cell_px']))
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 20)

paused = False
tick = 0
diag_last = None
running = True

# -----------------------------
# 5) One simulation tick
# -----------------------------
def step_once(C: np.ndarray, F: np.ndarray, tick: int):
    # (a) Rule step
    N8 = neigh8_sum(C)
    C_star = (alive_lut[N8] * C + dead_lut[N8] * (1 - C)).astype(np.uint8)

    # (b) Hard food gate (every gate_every ticks): kill cells that WERE alive if local food < 0.1
    C_new = C_star
    if CFG['gate_every'] >= 1 and (tick % int(CFG['gate_every']) == 0):
        F_local = mean3x3(F) if CFG['gate_on_local_meanF'] else F
        kill_mask = (C == 1) & (F_local < 0.1)
        if np.any(kill_mask):
            C_new = C_new.copy()
            C_new[kill_mask] = 0

    # (c) No central seeding / spawning in manual mode
    if (not CFG["manual_mode"]) and (CFG['seed_zone_D'] > 0):
        r0 = H//2 - CFG['seed_zone_D']//2
        c0 = W//2 - CFG['seed_zone_D']//2
        Cc = C_new[r0:r0+CFG['seed_zone_D'], c0:c0+CFG['seed_zone_D']]
        p_seed = CFG['p_seed_warmup'] if tick < CFG['warmup_ticks'] else CFG['p_seed_base']
        if p_seed > 0.0:
            Cc |= (rng.random(Cc.shape, dtype=np.float32) < p_seed).astype(np.uint8)

    if (not CFG["manual_mode"]) and (CFG['food_spawn_prob'] > 0.0):
        Us = rng.random((H, W), dtype=np.float32)
        F = np.where(Us < CFG['food_spawn_prob'], CFG['food_spawn_value'], F)

    # (d) Diffusion (explicit, conservative)
    rho = mean3x3(C_new).astype(np.float32)
    D_cell = (CFG['D_dead'] + (CFG['D_alive'] - CFG['D_dead']) * rho).astype(np.float32)
    D_right = np.roll(D_cell, -1, axis=1)
    D_down  = np.roll(D_cell, -1, axis=0)
    Dx = hmean(D_cell, D_right)
    Dy = hmean(D_cell, D_down)
    if CFG['enforce_cfl_assert']:
        assert_cfl(Dx, Dy, dt)

    F_right = np.roll(F, -1, axis=1)
    F_down  = np.roll(F, -1, axis=0)
    Jx = -Dx * (F_right - F)
    Jy = -Dy * (F_down  - F)
    divJ = (Jx - np.roll(Jx, 1, axis=1)) + (Jy - np.roll(Jy, 1, axis=0))
    F_tilde = (F - dt * divJ).astype(np.float32)

    # (e) Consumption + leak + clamp
    F_new = F_tilde * np.exp(-CFG['mu'] * C_new * dt, dtype=np.float32) - CFG['lambda_leak'] * F_tilde * dt
    F_new = np.clip(F_new.astype(np.float32), CFG['F_min'], CFG['F_max'])

    # Diagnostics
    diag = dict(
        flip_rate=float(np.mean(C_new != C_star)),
        live_mass=int(C_new.sum()),
        total_food=float(F_new.sum()),
    )
    return C_new, F_new, diag

# -----------------------------
# 6) Main loop
# -----------------------------
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
            elif event.key == pygame.K_SPACE:
                paused = not paused

    if not paused:
        for _ in range(CFG['ticks_per_render']):
            C, F, diag_last = step_once(C, F, tick)
            tick += 1
            if (tick % CFG['log_every_ticks'] == 0) and diag_last is not None:
                print(f"[t={tick:6d}] live={diag_last['live_mass']:7d} | "
                      f"food={diag_last['total_food']:.1f} | flips={diag_last['flip_rate']*100:.3f}%")

    frame = build_frame_surface(C, F)
    screen.blit(frame, (0, 0))

    if diag_last is not None:
        hud = (f"mode={CFG['mode']}  manual={CFG['manual_mode']}  pattern={CFG['pattern_name']}  "
               f"gate_every={CFG['gate_every']}  t={tick}  live={diag_last['live_mass']}  "
               f"food={diag_last['total_food']:.0f}  flips={diag_last['flip_rate']*100:.2f}%  "
               f"{'PAUSED' if paused else ''}")
        text = font.render(hud, True, (255, 255, 255))
        screen.blit(text, (8, 8))

    pygame.display.flip()
    clock.tick(CFG['target_fps'])

pygame.quit()
try:
    sys.exit(0)
except SystemExit:
    pass


pygame 2.6.0 (SDL 2.28.4, Python 3.10.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
[t=    60] live=      0 | food=239282.0 | flips=0.000%
[t=   120] live=      0 | food=238569.1 | flips=0.000%
[t=   180] live=      1 | food=237861.4 | flips=0.000%
[t=   240] live=      0 | food=237158.7 | flips=0.000%
[t=   300] live=      1 | food=236461.0 | flips=0.000%
[t=   360] live=     30 | food=235753.7 | flips=0.000%
[t=   420] live=     53 | food=235039.3 | flips=0.000%
[t=   480] live=    158 | food=234282.2 | flips=0.000%
[t=   540] live=    181 | food=233479.7 | flips=0.000%
[t=   600] live=     88 | food=232715.2 | flips=0.000%
[t=   660] live=     55 | food=232005.3 | flips=0.000%
[t=   720] live=     53 | food=231317.4 | flips=0.000%
[t=   780] live=     53 | food=230634.9 | flips=0.000%
[t=   840] live=     54 | food=229960.1 | flips=0.000%
[t=   900] live=     53 | food=229294.5 | flips=0.000%
[t=   960] live=     54 | food=228635.4 | flips=0.000%
[t=  1